In [ ]:
!pip install matplotlib mpmath numpy pandas sympy bokeh

In [ ]:
from google.colab import drive

drive.mount("/content/gdrive", force_remount=True)


Mounted at /content/gdrive


## For interactive Bokeh plot this code has to be inside the cell of the Bokeh code.

In [ ]:
from bokeh.io import output_notebook
output_notebook()

-------------------------------------------------------------------------------------------------------------------




---



**AFTER THIS POINT YOU CAN START WORKING**

---








Sure, here is a code you can use for the Bokeh server:

In [ ]:
import pandas as pd
from bokeh.plotting import figure, curdoc
from bokeh.models import Select, ColumnDataSource
from bokeh.layouts import column
from bokeh.transform import factor_cmap
from bokeh.palettes import Category20c

# Load the example data
data = pd.read_csv("bank.csv", delimiter=";")

# Create a ColumnDataSource and populate it with the entire DataFrame
source = ColumnDataSource(data=dict(
    marital=data['marital'],
    balance=data['balance'],
    y=data['y']
))

# Create a dropdown menu for choosing the value of the target variable
options = ['yes', 'no', 'all']
select_y = Select(title="Select Target Value:", value=options[0], options=options)

# Create the figure
p = figure(x_range=data['marital'].unique(), min_height=350, title="Average Balance by Marital Status",
           toolbar_location=None, tools="hover", tooltips="@marital: @balance{0.00}")

# Define a function to update the plot when the dropdown value changes
def update_plot(attrname, old, new):
    """
    This function updates the Bokeh bar plot with data based on the selected value of the target variable "y".
    It can show the balance for each marital status based on a subset of the values of the target variable "yes", "no", or "both".
    """
    selected_y = select_y.value

    # Filter the DataFrame based on the selected value
    if selected_y == 'all':
        df_selected_y = data[(data['y'] == 'yes') | (data['y'] == 'no')]
    else:
        df_selected_y = data[data['y'] == selected_y]

    # Calculate the average balance for each marital status
    avg_balance_by_marital = df_selected_y.groupby('marital')['balance'].mean().reset_index()

    # Sort the bars based on the average balance
    avg_balance_by_marital = avg_balance_by_marital.sort_values(by='balance', ascending=False)

    # Update the data for the plot
    source.data = dict(
        marital=avg_balance_by_marital['marital'],
        balance=avg_balance_by_marital['balance']
    )

# Call the update_plot function initially to populate the plot
update_plot(None, None, options[0])

# Add bars to the plot
colors = Category20c[len(data['marital'].unique())]
p.vbar(x='marital', top='balance', width=0.9, source=source, fill_color=factor_cmap('marital', palette=colors, factors=data['marital'].unique()))

# Attach the callback function to the dropdown menu
select_y.on_change('value', update_plot)

# Arrange the plot and the dropdown menu in a layout
layout = column(select_y, p)

# Add the layout to the current document (Bokeh server)
curdoc().add_root(layout)


You can save this code to a file named `app.py`, and then run it from the command-line like this:

`bokeh server --show app.py`